Piotr Pawłowski laboratorium 5

In [3]:
import numpy as np
import re
from sklearn.cluster import DBSCAN

## metryka cosinusowa

In [4]:
def cos_metric(vector1,vector2):
    if np.linalg.norm(vector1)!=0 and np.linalg.norm(vector2)!=0:
        return np.abs(1-(np.dot(vector1,vector2/(np.linalg.norm(vector1)*np.linalg.norm(vector2)))))
    return 0

In [5]:
print(cos_metric(np.array([5,0,3,0,2,0,0,2,0,0]),np.array([3,0,2,0,1,1,0,1,0,1])))

0.06439851429360033


## metryka euklidesowa

In [6]:
def eucl_metric(vector1,vector2):
    return np.sqrt(np.linalg.norm(vector1-vector2))

In [7]:
print(eucl_metric(np.array([5,0,3,0,2,0,0,2,0,0]),np.array([3,0,2,0,1,1,0,1,0,1])))

1.7320508075688772


## metryka dice

In [8]:
def dice_metric(vector1,vector2):
#     print(vector1[vector1==vector2])
    if np.linalg.norm(vector1)==0 and np.linalg.norm(vector2)==0:
        return 0
    return 1-(2*np.linalg.norm(vector1[vector1==vector2])/(np.linalg.norm(vector1)+np.linalg.norm(vector2)))

In [9]:
dice_metric(np.array([5,0,3,0,2,0,0,2,0,0]),np.array([3,0,2,0,1,1,0,1,0,1]))
v1=np.array([5,0,3,0,2,0,0,2,0,0])
v2=np.array([5,0,3,0,2,0,0,1,0,0])
print(v1[v1==v2])
print(dice_metric(v1,v2))

[5 0 3 0 2 0 0 0 0]
0.031189599309305205


In [10]:
def make_ngrams(word,n):
    x={}
    for i in range(len(word)-n+1):
        if not word[i:i+n] in x:
            x[word[i:i+n]]=1 
        else:
            x[word[i:i+n]]+=1
    return x
def make_ngrams_adress(x,n=3):
    return make_ngrams(tuple(re.split(" |,|\.",x)),n)

In [11]:
def make_vectors(ngram_x,ngram_y):
    keys=set(ngram_x.keys()).union(set(ngram_y.keys()))
    key_list=list(keys)
    x_v=np.zeros(len(key_list))
    y_v=np.zeros(len(key_list))
    for i,e in enumerate(key_list):
        if e in ngram_x:
            x_v[i]=ngram_x[e]
        if e in ngram_y:
            y_v[i]=ngram_y[e]
    return x_v,y_v

## Levensthein distance

In [12]:
def delta(a,b):
    return 0 if a==b else 1
def lev_dist(t1,t2):
    m=len(t1)
    n=len(t2)
    d=[[0 for i in range(n+1)] for i in range(m+1)]
    for i in range(m+1):
        d[i][0]=i
    for i in range(n+1):
        d[0][i]=i
    for i in range(1,m+1,1):
        for j in range(1,n+1,1):
            cost=delta(t1[i-1],t2[j-1])
            d[i][j]=min(d[i-1][j]+1,d[i][j-1]+1,d[i-1][j-1]+cost)
    ans=d[m][n]
    return ans

## pliki

In [13]:
adresses=[]
word_count={}
with open('lines.txt','r') as file:
    adresses=file.readlines()
    for adress in adresses:
        for word in adress.split():
            if word_count.get(word,0) == 0:
                word_count[word] = 1
            else:
                word_count[word] = word_count[word]+1

avg=sum(list(word_count.values()))/len(word_count)

stoplist=[]
# tworzenie stoplisty
for word,count in word_count.items():
    if count>avg*4:
        stoplist.append(word)

## macierz odległości

In [14]:
def calc_d_matrix(metric,texts_ngrams,metric_type=''):
    result=[[0 for i in range(len(texts_ngrams))] for _ in range(len(texts_ngrams)) ]
    for i in range(len(texts_ngrams)):
        for j in range(len(texts_ngrams)):
            if metric_type!='lev':
                x,y=make_vectors(texts_ngrams[i],texts_ngrams[j]) 
            else:
                x,y=texts_ngrams[i],texts_ngrams[j]
            result[i][j] = metric(x,y)
    return result

In [15]:
adresses=adresses[:200]
ngrams=[]
for adr in adresses:
    ngrams.append(make_ngrams_adress(adr))

klasteryzacja

In [16]:
d_matrix_cos=calc_d_matrix(cos_metric,ngrams)

d_matrix_euclid=calc_d_matrix(eucl_metric,ngrams)

d_matrix_dice=calc_d_matrix(dice_metric,ngrams)


In [17]:
d_matrix_lev=calc_d_matrix(lev_dist,adresses,'lev')

In [18]:
def make_etiquetes(labels,data):
    labeled_adr=[[] for i in range(len(set(labels)))]
    for n in labels:
        labeled_adr[n].append(data[n])
    return labeled_adr

In [19]:
dice_clusters_m=DBSCAN(eps=0.4,min_samples=1,metric='precomputed')
cos_clusters_m=DBSCAN(eps=0.2,min_samples=1,metric='precomputed')
eucl_clusters_m=DBSCAN(eps=1.8,min_samples=1,metric='precomputed')
lev_clusters_m=DBSCAN(eps=11,min_samples=1,metric='precomputed')

dice_fitted=dice_clusters_m.fit(d_matrix_dice).labels_
cos_fitted=cos_clusters_m.fit(d_matrix_cos).labels_
eucl_fitted=eucl_clusters_m.fit(d_matrix_euclid).labels_
lev_fitted=lev_clusters_m.fit(d_matrix_lev).labels_

clusters_dice=make_etiquetes(dice_fitted,adresses)
clusters_cos=make_etiquetes(cos_fitted,adresses)
clusters_eucl=make_etiquetes(eucl_fitted,adresses)
clusters_lev=make_etiquetes(lev_fitted,adresses)


In [29]:
for cluster in clusters_dice:
    print('cluster:')
    print(cluster)

cluster:
['/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA\n']
cluster:
["''PA INTERIOR'' LTD BOLSHAYA LUBYANKA STREET, 16/4 MOSCOW, 101000, RUSSIA INN/KPP 7704550148//770801001 495-984-8611\n"]
cluster:
["''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669\n"]
cluster:
["''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--\n"]
cluster:
["''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND\n"]
cluster:
["'MASTER PLUS CO.,LTD.' 143000,RUSSIA,MO,ODINSOVO, MOJAISKOE, SHOSSE,153G TEL:+7495 7273939\n"]
cluster:
['"2TIGERS GROUP LIMITED"  ROOM 504 JINSHAZHOU SHANGSHUI ROAD,  GUANGZHOU 510160\n']
cluster:
['"ALDETRANS" LLC, 105066, MOSCOW, RUSSIA, TOKMAKOV LANE, 11. TEL:+7(495)641-03-89\n', '"ALDETRANS" LLC, 105066, MOSCOW, RUSSIA, TOKMAKOV LANE, 11. TEL:+7(495)641-03-89\n', '"ALDETRANS" LLC, 105

klasteryzacja ze stoplistą

In [21]:
stop_applied=[]
def del_stoplist(string,stoplist):
    res=[x for x in string.split() if not x in stoplist]
    return res

In [22]:
for adr in adresses:
    stop_applied.append(' '.join(del_stoplist(adr,stoplist)))
ngrams=[]
# print(stop_applied)
for adr in stop_applied:
    ngrams.append(make_ngrams_adress(adr))

In [23]:
d_matrix_cos=calc_d_matrix(cos_metric,ngrams)

d_matrix_euclid=calc_d_matrix(eucl_metric,ngrams)

d_matrix_dice=calc_d_matrix(dice_metric,ngrams)

In [24]:
d_matrix_lev=calc_d_matrix(lev_dist,stop_applied,'lev')

In [25]:
dice_clusters_m=DBSCAN(eps=0.4,min_samples=1,metric='precomputed')
cos_clusters_m=DBSCAN(eps=0.2,min_samples=1,metric='precomputed')
eucl_clusters_m=DBSCAN(eps=1.8,min_samples=1,metric='precomputed')
lev_clusters_m=DBSCAN(eps=11,min_samples=1,metric='precomputed')

dice_fitted=dice_clusters_m.fit(d_matrix_dice).labels_
cos_fitted=cos_clusters_m.fit(d_matrix_cos).labels_
eucl_fitted=eucl_clusters_m.fit(d_matrix_euclid).labels_
lev_fitted=lev_clusters_m.fit(d_matrix_lev).labels_

clusters_dice_with_sl=make_etiquetes(dice_fitted,adresses)
clusters_cos_with_sl=make_etiquetes(cos_fitted,adresses)
clusters_eucl_with_sl=make_etiquetes(eucl_fitted,adresses)
clusters_lev_with_sl=make_etiquetes(lev_fitted,adresses)


In [28]:
for cluster in clusters_dice_with_sl:
    print('cluster:')
    print(cluster)

cluster:
['/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA\n']
cluster:
["''PA INTERIOR'' LTD BOLSHAYA LUBYANKA STREET, 16/4 MOSCOW, 101000, RUSSIA INN/KPP 7704550148//770801001 495-984-8611\n"]
cluster:
["''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669\n"]
cluster:
["''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--\n"]
cluster:
["''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND\n"]
cluster:
["'MASTER PLUS CO.,LTD.' 143000,RUSSIA,MO,ODINSOVO, MOJAISKOE, SHOSSE,153G TEL:+7495 7273939\n"]
cluster:
['"2TIGERS GROUP LIMITED"  ROOM 504 JINSHAZHOU SHANGSHUI ROAD,  GUANGZHOU 510160\n']
cluster:
['"ALDETRANS" LLC, 105066, MOSCOW, RUSSIA, TOKMAKOV LANE, 11. TEL:+7(495)641-03-89\n']
cluster:
['"A-LIFT",JSC 1 PROSPEKT MARSHALA ZHUKOVA,MOSCOW 123308,RUSSIA  T: +7(495)784-7961\n']
cluster:
[